In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


In [8]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints

def split_by_threshold(seq, max_diff=50):  
    it = iter(seq)
    last = next(it)
    part = [last]

    for curr in it:
        if curr - last > max_diff:
            yield part
            part = []

        part.append(curr)
        last = curr
#         print(part)
        
    yield part
    
def classify_by_offset(points):
    """
    Put logic for classifying an offset list here
    """
    #
    offset_difference_list = points['Time_Entry_Intervals']

    alt_list = offset_difference_list[::2]

    if len(set(offset_difference_list)) == 1 and len(offset_difference_list) > 1:
        points["Presentation_Type"] == "PEN"
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
    elif (len(offset_difference_list) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_difference_list) >= 3):
        points["Presentation_Type"] == 'ID'
    elif len(offset_difference_list) >= 1:
        points["Presentation_Type"] == 'Fuga'


In [3]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')


Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei


In [198]:
#  This works, but it produces LONG lists of related entries
# Includes all the offset differences and time/interval data
# but we need to break up the long series around the gaps

piece = importScore('https://crimproject.org/mei/CRIM_Model_0015.mei')

nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=5)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=5)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()


#  now build up metata for each group of similar soggetti
# stack the similar soggetti
# extract list of voices
# extract list of starting offsets
# get time ints
# extract starting pitches for each
# get entry ints
# make a 'row' with offsets, voices, time ints and entry ints
# add that row to DF

points = pd.DataFrame()
for matches in full_list_of_matches["match"]:
# for matches in test:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    voice_list = entry_array['voice'].to_list()
    offset_list = entry_array.index.to_list()

    time_ints = numpy.diff(offset_list).tolist()
    tone_coordinates =  list(zip(offset_list, voice_list))
    mel_ints = find_entry_int_distance(tone_coordinates, piece)
    first_offset = offset_list[0]
#     first_note_list = nr[offset_list]
    meas_beat = det[det.index.get_level_values('Offset').isin(offset_list)]
    mb2 = meas_beat.reset_index()
    mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
    meas_beat_list = mb2['mb'].to_list()


    temp = {'First_Offset': first_offset, 
            'Offsets': offset_list, 
            'Measures_Beats': meas_beat_list,
            "Soggetti": matches,
            'Voices': voice_list, 
            'Time_Entry_Intervals': time_ints, 
            'Melodic_Entry_Intervals': mel_ints}
    temp
    points = points.append(temp, ignore_index=True).sort_values("First_Offset")
points_no_singles = points[points['Offsets'].apply(len) > 1]
points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


# points_no_singles = points_no_singles['Soggetti'].apply(lambda x: ('\n'.join(x)))

points_no_singles.head()



Memoized piece detected.


/var/folders/k5/k4j_550s2yq5776vgmw6r_xr0000gp/T/ipykernel_62534/4282952611.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices
0,0.0,"[1/1.0, 3/1.0, 10/1.0, 12/2.0]","[P-4, P-5, P-4]","[0.0, 16.0, 72.0, 90.0]","[5, -2, 2, 2, 2, 4, -2, 2, 2, 2]","[16.0, 56.0, 18.0]","[Superior, Contratenor, Tenor, Bassus]"
2,38.0,"[5/4.0, 7/2.0, 7/4.0, 10/4.0, 14/4.0, 16/4.0, ...","[P-5, P-4, P8, P-8, P5, P4, P-8]","[38.0, 50.0, 54.0, 78.0, 110.0, 126.0, 138.0, ...","[-3, 2, -2, -2, 2, -3, 2, -2, -2, 1]","[12.0, 4.0, 24.0, 32.0, 16.0, 12.0, 8.0]","[Superior, Contratenor, Bassus, Superior, Teno..."
5,174.0,"[22/4.0, 24/2.0, 26/4.0, 28/2.0, 154/2.0, 154/...","[P5, P-12, P5, P8, P-8, M-2, P-4, P12, P-12]","[174.0, 186.0, 206.0, 218.0, 1234.0, 1238.0, 1...","[3, -2, -2, -2, -3, 3, -2, -2, -2, -2]","[12.0, 20.0, 12.0, 1016.0, 4.0, 16.0, 4.0, 12....","[Contratenor, Superior, Bassus, Tenor, Superio..."
10,266.0,"[34/2.0, 38/2.0, 40/2.0]","[P1, P-5]","[266.0, 298.0, 314.0]","[2, -3, 5, 1, -5]","[32.0, 16.0]","[Bassus, Tenor, Bassus]"
11,274.0,"[35/2.0, 36/2.0, 41/2.0]","[P-5, P1]","[274.0, 282.0, 322.0]","[2, -3, 5, 1, -2]","[8.0, 40.0]","[Superior, Contratenor, Tenor]"


In [178]:
# TEST works up to new_offset_list
# that is, it uses the 'split_by_threshold' function to break up the long sets
# but now we need to use those short sets to produce short 
piece = importScore('https://crimproject.org/mei/CRIM_Model_0015.mei')

nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=5)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=5)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()


#  now build up metata for each group of similar soggetti
# stack the similar soggetti
# extract list of voices
# extract list of starting offsets
# get time ints
# extract starting pitches for each
# get entry ints
# make a 'row' with offsets, voices, time ints and entry ints
# add that row to DF

points = pd.DataFrame()
new_offset_list = []
    
for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
#     voice_list = entry_array['voice'].to_list()
    offset_list = entry_array.index.to_list()
    short_list = list(split_by_threshold(offset_list))
    for item in short_list:
        new_offset_list.append(item)
print(entry_array, item)
    
# related_entry_list   

#     time_ints = numpy.diff(new_offset_list)
#     tone_coordinates =  list(zip(new_offset_list, voice_list))
# #     tone_coordinates.ffill(inplace=True)
#     mel_ints = find_entry_int_distance(tone_coordinates, piece)
#     first_offset = offset_list[0]
# #     first_note_list = nr[offset_list]
#     meas_beat = det[det.index.get_level_values('Offset').isin(offset_list)]
#     mb2 = meas_beat.reset_index()
#     mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
#     meas_beat_list = mb2['mb'].to_list()


#     temp = {'First_Offset': first_offset, 
#             'Offsets': offset_list, 
#             'Measures_Beats': meas_beat_list,
#             "Soggetti": matches,
#             'Voices': voice_list, 
#             'Time_Entry_Intervals': time_ints, 
#             'Melodic_Entry_Intervals': mel_ints}
#     temp
#     points = points.append(temp, ignore_index=True).sort_values("First_Offset")
# points_no_singles = points[points['Offsets'].apply(len) > 1]
# points_no_singles.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)


# # points_no_singles = points_no_singles['Soggetti'].apply(lambda x: ('\n'.join(x)))

# points_no_singles.head()



Memoized piece detected.
             voice         pattern
798.0       Bassus  5, 1, -3, 2, 2
826.0  Contratenor  5, 1, -3, 2, 2
838.0     Superior  5, 1, -3, 2, 2
850.0        Tenor  4, 1, -3, 2, 2
866.0       Bassus  4, 1, -3, 2, 2 [826.0, 838.0, 850.0, 866.0]


In [4]:
piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')


Memoized piece detected.


In [5]:
# THIS WORKS 3/1/22
# piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')
points = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=5)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=5)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
#     voice_list = entry_array['voice'].to_list()
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    for item in split_list:
#         print(item)
        array = entry_array[entry_array.index.get_level_values(0).isin(item)]
        short_offset_list = array.index.to_list()
        time_ints = numpy.diff(array.index).tolist()
        voice_list = array['voice'].to_list()
# time_ints
        tone_coordinates =  list(zip(short_offset_list, voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        first_offset = short_offset_list[0]
# #     first_note_list = nr[offset_list]
        meas_beat = det[det.index.get_level_values('Offset').isin(short_offset_list)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()


        temp = {'First_Offset': first_offset, 
                    'Offsets': short_offset_list, 
                    'Measures_Beats': meas_beat_list,
                    "Soggetti": matches,
                    'Voices': voice_list, 
                    'Time_Entry_Intervals': time_ints, 
                    'Melodic_Entry_Intervals': mel_ints}
        
        points = points.append(temp, ignore_index=True).sort_values("First_Offset")
        points = points[points['Offsets'].apply(len) > 1]
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
#     points_clean

# points_no_singles = points_clean['Soggetti'].apply(lambda x: ('\n'.join(x)))
points
# temp
          

,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices
0,0.0,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]","[P-8, P1, P-8]","[0.0, 16.0, 32.0, 48.0]","[4, 1, 2, 2, -3]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]"
1,56.0,"[8/1.0, 10/1.0, 12/1.0]","[P-8, P1]","[56.0, 72.0, 88.0]","[-2, -2, -2, 2, -2]","[16.0, 16.0]","[[Superius], Altus, Tenor]"
2,124.0,"[16/3.0, 18/3.0, 20/3.0, 22/3.0]","[P-8, P1, P-8]","[124.0, 140.0, 156.0, 172.0]","[1, 1, 2, 2, -3]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]"
3,176.0,"[23/1.0, 28/1.0]",[P-8],"[176.0, 216.0]","[-3, -2, 2, 2, 2]",[40.0],"[Altus, Bassus]"
4,244.0,"[31/3.0, 35/3.0]",[P-8],"[244.0, 276.0]","[1, 2, 2, 1, 2, 1, 1, 2, 1, 2]",[32.0],"[[Superius], Tenor]"
5,316.0,[40/3.0],[M-6],"[316.0, 316.0]","[1, 1, 2, -2, -2]",[0.0],"[[Superius], Tenor]"
6,428.0,"[54/3.0, 55/1.0, 59/3.0, 60/1.0]","[P-5, P-4, P-5]","[428.0, 432.0, 468.0, 472.0]","[4, -2, 2, 2, -3]","[4.0, 36.0, 4.0]","[[Superius], Altus, Tenor, Bassus]"
7,508.0,"[64/3.0, 66/3.0, 67/3.0, 69/3.0]","[P-5, P-4, P-5]","[508.0, 524.0, 532.0, 548.0]","[2, -3, 2, -3, 2]","[16.0, 8.0, 16.0]","[[Superius], Altus, Tenor, Bassus]"
8,616.0,"[78/1.0, 81/1.0]",[P-8],"[616.0, 640.0]","[1, 1, 2, -3, 2]",[24.0],"[[Superius], Tenor]"
9,748.0,"[94/1.0, 98/1.0]",[P1],"[748.0, 796.0]","[1, 1, 2, -2, -2]",[48.0],"[Altus, Altus]"


In [40]:
test = points.head(10)
offset_difference_list = test['Time_Entry_Intervals'][0]

In [41]:
alt_list = offset_difference_list[::2]
if len(set(offset_difference_list)) == 1 and len(offset_difference_list) > 1:
#     points["Presentation_Type"] == "PEN"
    print('PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_difference_list) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_difference_list) >= 3):
#     points["Presentation_Type"] == 'ID'
    print('ID')
elif len(offset_difference_list) >= 1:
#     points["Presentation_Type"] == 'Fuga'
    print('FUGA')

PEN


TypeError: list indices must be integers or slices, not str

In [42]:
points['Presentation_Type'] = points.apply(lambda row: classify_by_offset(row['Time_Entry_Intervals']),axis=1)


TypeError: list indices must be integers or slices, not str

In [8]:
# Durations of Notes in Ngrams
#  KEEP THIS!
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
ng = piece.getNgrams(df=dur, n=5)
ng

,Superior,Contratenor,Tenor,Bassus
0.0,"(4.0, 6.0, 2.0, 2.0, 2.0)","(16.0, 4.0, 6.0, 2.0, 2.0)","(72.0, 4.0, 6.0, 2.0, 2.0)","(54.0, 2.0, 2.0, 4.0, 2.0)"
4.0,"(6.0, 2.0, 2.0, 2.0, 3.0)",NaN,NaN,NaN
10.0,"(2.0, 2.0, 2.0, 3.0, 0.5)",NaN,NaN,NaN
12.0,"(2.0, 2.0, 3.0, 0.5, 0.5)",NaN,NaN,NaN
14.0,"(2.0, 3.0, 0.5, 0.5, 2.0)",NaN,NaN,NaN
...,...,...,...,...
1316.0,NaN,"(1.0, 1.0, 2.0, 2.0, 2.0)","(3.0, 1.0, 2.0, 2.0, 4.0)","(2.0, 1.0, 1.0, 1.0, 1.0)"
1317.0,NaN,"(1.0, 2.0, 2.0, 2.0, 4.0)",NaN,NaN
1318.0,NaN,"(2.0, 2.0, 2.0, 4.0, 16.0)",NaN,"(1.0, 1.0, 1.0, 1.0, 2.0)"
1319.0,NaN,NaN,"(1.0, 2.0, 2.0, 4.0, 16.0)","(1.0, 1.0, 1.0, 2.0, 4.0)"
